# MSE损失函数

In [1]:
import torch
from torch.nn import MSELoss#类
yhat = torch.randn(size=(50,),dtype=torch.float32)
y = torch.randn(size=(50,),dtype=torch.float32)
criterion =MSELoss() #实例化
loss = criterion(yhat,y)
loss #没有设置随机数种子，所以每次运行的数字都会不一致
#在MSELoss中有重要的参数，reduction
#当reduction = "mean" (默认也是mean)，则输出MSE
#当reduction = "sum"，则输出SSE
criterion = MSELoss(reduction = "mean") #实例化
criterion(yhat,y)
criterion = MSELoss(reduction = "sum")
criterion(yhat,y)

tensor(105.2740)

In [2]:
yhat

tensor([ 1.2856,  0.4419, -0.5217, -1.3790,  0.9287, -1.1419, -0.6497, -1.3393,
         0.6949,  1.8447, -0.7083, -2.5326,  1.4791,  0.9352, -0.7228,  0.4340,
        -0.3933, -0.3710,  0.6111,  0.3316, -0.2658, -1.6776, -0.0571, -0.4329,
        -0.4885,  0.7979, -0.7102, -1.0926, -0.4037, -0.8492, -0.6113,  0.8313,
        -0.6225,  2.5553, -0.5076, -0.6394, -0.6878,  0.4487,  3.0216,  1.9826,
         0.9998,  1.0153, -0.3754,  1.0142, -0.8512, -1.0406, -0.9298, -0.9853,
         2.1803,  0.4245])

In [3]:
y

tensor([ 0.5683,  0.4219,  0.0042,  0.8480, -0.7894,  0.4232,  1.5119, -0.6371,
         0.2657, -2.1157,  0.5179, -0.2854,  0.3298, -0.8231, -0.1034,  0.1760,
        -0.2353,  0.6692, -1.2651, -0.7630, -0.0075, -0.7563, -0.3748,  0.5689,
         0.3061,  1.1553, -1.3420,  0.3253, -0.4502,  0.9837, -0.4306,  1.5947,
        -0.5605, -0.0958,  0.6806,  1.0936, -1.2114,  0.8631, -0.3167, -0.4117,
         0.3595,  0.6218, -0.5909,  0.2274, -0.0438,  1.6160, -0.9271, -1.3443,
        -0.0315,  2.4117])

# 交叉熵损失函数

## 极大似然估计
如果一个事件的发生概率很大，那这个事件应该很容易发生。相应的，如果依赖于权重 的任意事
件的发生就是我们的目标，那我们只要寻找令其发生概率最大化的权重 就可以了。寻找相应的权
重 ，使得目标事件的发生概率最大，就是极大似然估计的基本方法。

其步骤如下：
1、构筑似然函数 ，用于评估目标事件发生的概率，该函数被设计成目标事件发生时，概率最
大
2、对整体似然函数取对数，构成对数似然函数
3、在对数似然函数上对权重 求导，并使导数为0，对权重进行求解

## 交叉熵损失函数


In [5]:
import torch
import time
N = 3*pow(10,3)
torch.random.manual_seed(420)
X = torch.rand((N,4),dtype=torch.float32)
w = torch.rand((4,1),dtype=torch.float32,requires_grad=True)
y = torch.randint(low=0,high=2,size=(N,1),dtype=torch.float32)
zhat = torch.mm(X,w)
sigma = torch.sigmoid(zhat)
Loss = -(1/N)*torch.sum((1-y)*torch.log(1-sigma)+y*torch.log(sigma)) # sum后求均值
Loss

tensor(0.7962, grad_fn=<MulBackward0>)

In [8]:
start = time.time() #捕获现在的时间
loss1 = -(1/N)*sum(y*torch.log(sigma) + (1-y)*torch.log(1-sigma))
now = time.time() #以秒计
print(now - start)

0.04208183288574219


In [9]:
start = time.time() # 使用torch库进行运算
loss2 = -(1/N)*torch.sum(y*torch.log(sigma) + (1-y)*torch.log(1-sigma))
now = time.time() #以秒计
print(now - start)

0.0027441978454589844


### torch库调用BCE

In [12]:
import torch.nn as nn
criterion = nn.BCELoss(reduction = "mean") #实例化
loss = criterion(sigma,y)
loss

tensor(0.7962, grad_fn=<BinaryCrossEntropyBackward>)

In [13]:
criterion2 = nn.BCEWithLogitsLoss(reduction = "mean") # 使用sigma激活之前输出
loss = criterion2(zhat, y)
loss

tensor(0.7962, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)

In [14]:
criterion2 = nn.BCEWithLogitsLoss(reduction = "mean") # 使用sigma激活之前输出，精度更好
loss = criterion2(sigma, y)
loss

tensor(0.7600, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)

## 多分类交叉熵

In [15]:
import torch
import torch.nn as nn
N = 3*pow(10,2)
torch.random.manual_seed(420)
X = torch.rand((N,4),dtype=torch.float32)
w = torch.rand((4,3),dtype=torch.float32,requires_grad=True)

#定义y时应该怎么做？应该设置为矩阵吗？
y = torch.randint(low=0,high=3,size=(N,),dtype=torch.float32)
zhat = torch.mm(X,w)
#从这里开始调用softmax和NLLLoss
logsm = nn.LogSoftmax(dim=1) #实例化
logsigma = logsm(zhat)
criterion = nn.NLLLoss() #实例化，转换为onehot
#由于交叉熵损失需要将标签转化为独热形式，因此不接受浮点数作为标签的输入
#对NLLLoss而言，需要输入logsigma
criterion(logsigma,y.long())

tensor(1.1591, grad_fn=<NllLossBackward>)

In [17]:
## 使用torch
criterion = nn.CrossEntropyLoss()
criterion(zhat,y.long())

tensor(1.1591, grad_fn=<NllLossBackward>)